# CL Test

## Installation
Install all the necessary library.

In [ ]:
! pip install avalanche-lib==0.3.1
! pip install micromind

In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam

import numpy as np
import matplotlib.pyplot as plt

from avalanche.benchmarks.classic import SplitMNIST
from avalanche.models import SimpleMLP
from avalanche.training.storage_policy import ReservoirSamplingBuffer


from joint_training import JointTraining
from fine_tuning import FineTuning
from comulative import Comulative
from replay import Replay
from latent_replay import LatentReplay
from phinet_v2 import PhiNet_v2

from micromind import PhiNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

torch.manual_seed(0)

c:\Users\matte\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu


## Benchmark

In [2]:
split_mnist = SplitMNIST(n_experiences=5, seed=0, return_task_id = True)

# recovering the train and test streams
train_stream = split_mnist.train_stream
test_stream = split_mnist.test_stream

## Training

### Training Parameters

In [3]:
criterion = nn.CrossEntropyLoss()

# Define other necessary parameters
train_mb_size = 128
eval_mb_size = 128

### Fine Tuning Strategy

In [ ]:
#model1 = SimpleMLP(num_classes=10).to(device)
model1 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=4 ,include_top = True, num_classes = 10).to(device)

#optimizer1 = SGD(model1.parameters(), lr=0.01, momentum=0.9)
optimizer1 = Adam(model1.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

fine_tuning = FineTuning(
    model=model1,
    optimizer=optimizer1,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

fine_tuning.train(train_stream)
fine_tuning.test(test_stream)

### Joint Training Strategy

In [ ]:
#model2 = SimpleMLP(num_classes=10).to(device)
model2 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=4 ,include_top = True, num_classes = 10).to(device)

#optimizer2 = SGD(model2.parameters(), lr=0.01, momentum=0.9)
optimizer2 = Adam(model2.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

joint_training = JointTraining(
    model=model2,
    optimizer=optimizer2,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

joint_training.train(train_stream)
joint_training.test(test_stream)

### Comulative Strategy

In [ ]:
#model3 = SimpleMLP(num_classes=10).to(device)
model3 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=4 ,include_top = True, num_classes = 10).to(device)

#optimizer3 = SGD(model3.parameters(), lr=0.01, momentum=0.9)
optimizer3 = Adam(model3.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

comulative = Comulative(
    model=model3,
    optimizer=optimizer3,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    device=device
)

comulative.train(train_stream)
comulative.test(test_stream)

### Replay Strategy

In [ ]:
#model4 = SimpleMLP(num_classes=10).to(device)
model4 = PhiNet(input_shape = (1, 28, 28), alpha = 0.5, beta = 1, t_zero = 6,num_layers=4 ,include_top = True, num_classes = 10).to(device)

#optimizer4 = SGD(model4.parameters(), lr=0.01, momentum=0.9)
optimizer4 = Adam(model4.parameters(), lr=0.01, weight_decay=0)

storage_p = ReservoirSamplingBuffer(max_size=1500)

train_epochs = 4

replay = Replay(
    model=model4,
    optimizer=optimizer4,
    criterion=criterion,
    train_mb_size=train_mb_size,
    train_epochs=train_epochs,
    eval_mb_size=eval_mb_size,
    storage_policy = storage_p,
    device=device
)

replay.train(train_stream)
replay.test(test_stream)

### Latent Replay Strategy

In [7]:
from avalanche.evaluation.metrics import ElapsedTime
import time
model4 = PhiNet_v2(pretrained="./TestModel/7_Layers/Adam.pth", device = device, num_layers= 7, latent_layer_num=9).to(device)

optimizer4 = Adam(model4.parameters(), lr=0.01, weight_decay=0)

train_epochs = 4

t = ElapsedTime()

latent_replay = LatentReplay(
    model=model4,
    optimizer=optimizer4,
    criterion=criterion,
    train_mb_size=21,
    replay_mb_size = 107,
    train_epochs=train_epochs,
    eval_mb_size=128,
    rm_size = 1500,
    device=device
)

latent_replay.train(train_stream)
t.update()

latent_replay.test(test_stream, Plotting=False)
t.update()

print(f"Inference time: {t.result():.2f} s")

PhiNet before: 
PhiNet(
  (_layers): ModuleList(
    (0): ZeroPad2d((0, 1, 0, 1))
    (1): SeparableConv2d(
      (_layers): ModuleList(
        (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): Conv2d(1, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(24, eps=0.001, momentum=0.999, affine=True, track_running_stats=True)
        (3): HSwish()
      )
    )
    (2): PhiNetConvBlock(
      (_layers): ModuleList(
        (0): Dropout2d(p=0.05, inplace=False)
        (1): DepthwiseConv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
        (2): BatchNorm2d(24, eps=0.001, momentum=0.999, affine=True, track_running_stats=True)
        (3): HSwish()
        (4): Conv2d(24, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (5): BatchNorm2d(12, eps=0.001, momentum=0.999, affine=True, track_running_stats=True)
      )
    )
    (3): PhiNetConvBlock(
      (_layers): ModuleList(
        (0): Conv